In [1]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Student_Mental_Health/Student_Mental_Health/research'

In [2]:
import os

In [3]:
os.chdir("../.")

In [4]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Student_Mental_Health/Student_Mental_Health'

In [5]:
# Entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataloadingConfig:
    root_dir: Path
    data_path: Path
    X_train: Path
    X_test: Path
    y_train: Path
    y_test: Path
    test_size: float

In [7]:
# for config manger
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.data_root])
    
    def get_data_loading_config(self) -> DataloadingConfig:
        config = self.config.data_loading
        params = self.params.test_size_args
        create_directories([config.root_dir])
        data_loading_config = DataloadingConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            X_train = config.X_train,
            y_train = config.y_train,
            X_test = config.X_test,
            y_test = config.y_test,
            test_size = params.test_size,
        )
        return data_loading_config

In [9]:
import os
import zipfile
import urllib.request as request
import pandas as pd
from src.logging import logger
from src.componnents.data_ingestion import DataIngestion
from src.entity import DataIngestionConfig
from sklearn.model_selection import train_test_split

In [16]:
import os
class DataLoading:
    def __init__(self, config: DataloadingConfig):
        self.config = config

    def get_data_loading(self):
        config2 = DataIngestionConfig(root_dir=self.config.root_dir, data_path=self.config.data_path)
        data_ingestion = DataIngestion(config2)
        df = data_ingestion.get_data()
        return df
    
    def split_data(self):
        df = self.get_data_loading()
        X = df.drop('depression',axis=1)
        y = df['depression']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.config.test_size, random_state=42)
        return X_train, X_test, y_train, y_test

    def save_data(self):
        X_train, X_test, y_train, y_test = self.split_data()
        # if file exists skip else create it
        if not os.path.exists(self.config.X_train) and not os.path.exists(self.config.y_train) and not os.path.exists(self.config.X_test) and not os.path.exists(self.config.y_test):
            X_train.to_csv(self.config.X_train, index = False)
            X_test.to_csv(self.config.X_test, index = False)
            y_train.to_csv(self.config.y_train, index = False)
            y_test.to_csv(self.config.y_test, index = False)
        else:
            print("Files already exist")


In [17]:
config_manager = ConfigurationManager()
data_loading_config = config_manager.get_data_loading_config()
data_loader = DataLoading(config=data_loading_config)
df = data_loader.get_data_loading()
data_loader.save_data()

[2024-08-28 23:11:02,807: INFO: common: Yaml file: config/config.yaml loaded successfully]
[2024-08-28 23:11:02,809: INFO: common: Yaml file: params.yaml loaded successfully]
[2024-08-28 23:11:02,809: INFO: common: created directory at: data]
[2024-08-28 23:11:02,810: INFO: common: created directory at: data/data_loading]
   gender  age university   degree_level      degree_major academic_year  \
0    Male   20         PU  Undergraduate      Data Science      2nd year   
1    Male   20        UET   Postgraduate  Computer Science      3rd year   
2    Male   20       FAST  Undergraduate  Computer Science      3rd year   
3    Male   20        UET  Undergraduate  Computer Science      3rd year   
4  Female   20        UET  Undergraduate  Computer Science      3rd year   

      cgpa residential_status campus_discrimination sports_engagement  ...  \
0  3.0-3.5         Off-Campus                    No         No Sports  ...   
1  3.0-3.5         Off-Campus                    No         1-3

In [12]:
from src.componnents.data_ingestion import DataIngestion
from src.entity import DataIngestionConfig

config = DataIngestionConfig(root_dir='data/data_ingestion', data_path='data/MentalHealthSurvey.csv')
data_ingestion = DataIngestion(config)

# Move the file if it hasn't been moved yet
# data_ingestion.move_file_to_data_ingestion()

# Read and display the data
df = data_ingestion.get_data()


   gender  age university   degree_level      degree_major academic_year  \
0    Male   20         PU  Undergraduate      Data Science      2nd year   
1    Male   20        UET   Postgraduate  Computer Science      3rd year   
2    Male   20       FAST  Undergraduate  Computer Science      3rd year   
3    Male   20        UET  Undergraduate  Computer Science      3rd year   
4  Female   20        UET  Undergraduate  Computer Science      3rd year   

      cgpa residential_status campus_discrimination sports_engagement  ...  \
0  3.0-3.5         Off-Campus                    No         No Sports  ...   
1  3.0-3.5         Off-Campus                    No         1-3 times  ...   
2  2.5-3.0         Off-Campus                    No         1-3 times  ...   
3  2.5-3.0          On-Campus                    No         No Sports  ...   
4  3.0-3.5         Off-Campus                   Yes         No Sports  ...   

  study_satisfaction  academic_workload   academic_pressure  \
0          